In [1]:
%load_ext google.cloud.bigquery

## Find matching images and signals data
Purpose: 
 1. Work through millions of AIS signals from ships in fishing industry
    - Create a baseline of normal behavior based on their fishing type
 2. Aggregate each ships signals into voyages (times at sea, not port)
 3. Determine each ships behaivor over its voyages compared to baseline
 4. Write out results for other functions to visualize and determine 
    which ships have operations that may indicate use of Human Trafficking 
Created on Thu Jun 25 11:33:33 2020

Data: 
    Dataset of 26 Million AIS signals over a 5 year period for a selected
    group of 350 ships.   
    
    URL: https://globalfishingwatch.org/data-download/datasets/public-training-data-v1

    Provider: Global Fishing Watch based on a partnership explained in the paper
        
    Terms of Use: https://globalfishingwatch.org/datasets-and-code/

@author: harry newton

In [32]:
#
# Import libraries
#

import pandas as pd
import numpy as np
import pandas_gbq

import calendar
from datetime import datetime, timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore",category=mpl.cbook.mplDeprecation)

In [3]:
#
# name the data sources in GBQ to be used

imageryData="'bigquery-public-data.cloud_storage_geo_index.sentinel_2_index`"
imageryVars="*"

aisData="`global-fishing-watch.paper_dark_fishing_fleets_in_north_korea.ais_positions_2018`"
aisVars="ssvid, lat, lon, timestamp, fishing"
print(aisData)

params={}
#Set bounding box based on AIS data available from source

params['extentLeft']=127.0
params['extentRight']=137.0
params['extentTop']=44.0
params['extentBottom']=36.0

#Set time window for last month of AIS data available from source
#earliestDate="2018-12-01"
#latestDate="2018-12-31"
params['likeDate']='2018-12-%'  #matches Dec 1 to Dec 31, just for a start
       
#Set max cloud cover to tolerate
params['maxCloudCover']=10.0     
print("params=", params)

`global-fishing-watch.paper_dark_fishing_fleets_in_north_korea.ais_positions_2018`
params= {'extentBottom': 36.0, 'likeDate': '2018-12-%', 'extentRight': 137.0, 'extentLeft': 127.0, 'extentTop': 44.0, 'maxCloudCover': 10.0}


# Pull the IMAGES in the bounding box for the time window

In [4]:
%%bigquery IMAGESET --params $params
SELECT * FROM `bigquery-public-data.cloud_storage_geo_index.sentinel_2_index`
WHERE  
north_lat <= @extentTop
AND south_lat >= @extentBottom
AND west_lon >= @extentLeft
AND east_lon <= @extentRight
AND sensing_time LIKE @likeDate
AND CAST(cloud_cover AS NUMERIC) <= @maxCloudCover

#Results are 146 cloud-free images in December in the region

In [5]:
print(IMAGESET.describe())
#print(IMAGESET)
IMAGESET.shape

        north_lat   south_lat    west_lon    east_lon    total_size
count  146.000000  146.000000  146.000000  146.000000  1.460000e+02
mean    41.011602   40.106705  130.860553  131.766958  4.509498e+08
std      1.983439    1.930902    2.710754    2.652596  2.767339e+08
min     36.139831   36.005338  127.064866  127.904567  1.226817e+07
25%     39.744763   38.754417  128.762441  129.120311  1.723041e+08
50%     41.531839   40.555566  130.197228  130.981658  5.034006e+08
75%     42.452781   41.457605  133.706663  133.948432  6.849637e+08
max     43.906912   43.264818  136.109931  136.354324  9.026043e+08


(146, 14)

In [33]:
#write IMAGESET to BQ table, 
#documentation at https://pandas-gbq.readthedocs.io/en/latest/writing.html
#pandas_gbq.to_gbq(
#    df, 'my_dataset.my_table', project_id=projectid, if_exists='fail',
#)
pandas_gbq.to_gbq(
    IMAGESET, 'jt_data.images_table', project_id='gwu-cap', if_exists='replace',
)


The existing table has a different schema. Please wait 2 minutes. See Google BigQuery issue #191



Load is 100.0% Complete




In [56]:
def fetch_matching_signals(x1, x2, y1, y2, t1, t2):
# find ais signals same day, within bounding box of x1,x2 and y1, y2
# reset times to text to match data

  q="""#standardSQL
  SELECT 
      ssvid, lat, lon, timestamp, fishing
    FROM
      `global-fishing-watch.paper_dark_fishing_fleets_in_north_korea.ais_positions_2018` 
    WHERE 
      lon BETWEEN {x1} AND {x2}
      AND lat BETWEEN {y1} AND {y2}
      AND timestamp >= '{t1}'
      AND timestamp < '{t2}'
      """.format(x1=x1,x2=x2,y1=y1,y2=y2,t1=t1,t2=t2)
  #print('q=',q)
  signals_df = pd.read_gbq(q, project_id='gwu-cap', verbose=False, dialect='standard')
  
  #print("returning signals_df of size: ",signals_df.shape)
  return(signals_df)

# Now, for each image find corresponding signals (tight on lat, lon and same day)
# Accumulate the results in SIGNALS, which has a row for each signal and information 
# on the matching image


In [57]:

SIGNALS=pd.DataFrame()
print("starting with SIGNALS df of shape:",SIGNALS.shape)
for idx in IMAGESET.index:  #later ...len(IMAGESET)):
  x1=IMAGESET.loc[idx,'west_lon']
  x2=IMAGESET.loc[idx,'east_lon']
  y1=IMAGESET.loc[idx,'south_lat']
  y2=IMAGESET.loc[idx,'north_lat']
    
  # time adjustments because formats are different between imagery & signals  
  t_str=IMAGESET.loc[idx,'sensing_time']
  dateTime=t_str[:-8].split("T")
  dT=dateTime[0]+" "+dateTime[1]
  date_time_obj = datetime.strptime(dT, '%Y-%m-%d %H:%M:%S')
  t1= date_time_obj - timedelta(hours=1, minutes=0)
  t2= date_time_obj + timedelta(hours=1, minutes=0)
  #print ("timeWindow:",t1,"-",t2)  
  t1=str(t1)
  t2=str(t2)
  #print ("timeWindow:",t1,"-",t2)  
    
  
  print("calling fetch_signals:",x1,x2,y1,y2,t1,t2)
  signals=fetch_matching_signals(x1, x2, y1, y2, t1, t2)
  signals['imagery_time']=t_str
  signals['south_lat']=IMAGESET.loc[idx,'south_lat']  
  signals['north_lat']=IMAGESET.loc[idx,'north_lat']
  signals['west_lon']=IMAGESET.loc[idx,'west_lon']
  signals['east_lon']=IMAGESET.loc[idx,'east_lon']
  signals['product_id']=IMAGESET.loc[idx,'product_id']
  signals['base_url']=IMAGESET.loc[idx,'base_url']
    
  SIGNALS=pd.concat([SIGNALS,signals])
  print("***SIGNALS:",SIGNALS.shape)
  

starting with SIGNALS df of shape: (0, 0)
timeWindow: 2018-12-01 01:16:13 - 2018-12-01 03:16:13
timeWindow: 2018-12-01 01:16:13 - 2018-12-01 03:16:13
calling fetch_signals: 130.16555163125204 131.48033157941597 39.635966469557026 40.64488962074263 2018-12-01 01:16:13 2018-12-01 03:16:13
***SIGNALS: (0, 12)
timeWindow: 2018-12-14 01:25:26 - 2018-12-14 03:25:26
timeWindow: 2018-12-14 01:25:26 - 2018-12-14 03:25:26
calling fetch_signals: 130.21416884809912 131.14868024590672 42.350449664022705 43.34628997721646 2018-12-14 01:25:26 2018-12-14 03:25:26
***SIGNALS: (12, 12)
timeWindow: 2018-12-15 00:56:09 - 2018-12-15 02:56:09
timeWindow: 2018-12-15 00:56:09 - 2018-12-15 02:56:09
calling fetch_signals: 133.83144950836405 135.11532505161873 39.65584265364618 40.650889000785384 2018-12-15 00:56:09 2018-12-15 02:56:09
***SIGNALS: (12, 12)
timeWindow: 2018-12-24 01:26:32 - 2018-12-24 03:26:32
timeWindow: 2018-12-24 01:26:32 - 2018-12-24 03:26:32
calling fetch_signals: 127.83225811076329 129.1138

***SIGNALS: (492, 12)
timeWindow: 2018-12-20 00:55:43 - 2018-12-20 02:55:43
timeWindow: 2018-12-20 00:55:43 - 2018-12-20 02:55:43
calling fetch_signals: 134.34907460025434 135.11857289735818 41.4620070915216 42.452727236501275 2018-12-20 00:55:43 2018-12-20 02:55:43
***SIGNALS: (492, 12)
timeWindow: 2018-12-31 01:15:39 - 2018-12-31 03:15:39
timeWindow: 2018-12-31 01:15:39 - 2018-12-31 03:15:39
calling fetch_signals: 132.56867847258383 133.3310203197753 41.43892576714181 42.44062705505493 2018-12-31 01:15:39 2018-12-31 03:15:39
***SIGNALS: (503, 12)
timeWindow: 2018-12-20 00:56:12 - 2018-12-20 02:56:12
timeWindow: 2018-12-20 00:56:12 - 2018-12-20 02:56:12
calling fetch_signals: 133.83183805889408 135.11532505161873 39.65584265364618 40.650889000785384 2018-12-20 00:56:12 2018-12-20 02:56:12
***SIGNALS: (503, 12)
timeWindow: 2018-12-02 00:46:38 - 2018-12-02 02:46:38
timeWindow: 2018-12-02 00:46:38 - 2018-12-02 02:46:38
calling fetch_signals: 135.7877840122856 136.26484688608292 37.852920

***SIGNALS: (1113, 12)
timeWindow: 2018-12-18 01:06:20 - 2018-12-18 03:06:20
timeWindow: 2018-12-18 01:06:20 - 2018-12-18 03:06:20
calling fetch_signals: 131.2678955749513 131.4687010104677 39.635966469557026 40.310813942583465 2018-12-18 01:06:20 2018-12-18 03:06:20
***SIGNALS: (1113, 12)
timeWindow: 2018-12-09 01:26:30 - 2018-12-09 03:26:30
timeWindow: 2018-12-09 01:26:30 - 2018-12-09 03:26:30
calling fetch_signals: 128.99964953836775 129.8129663973118 38.760022439724004 39.750357915781656 2018-12-09 01:26:30 2018-12-09 03:26:30
***SIGNALS: (1115, 12)
timeWindow: 2018-12-01 01:16:24 - 2018-12-01 03:16:24
timeWindow: 2018-12-01 01:16:24 - 2018-12-01 03:16:24
calling fetch_signals: 131.30052784636297 132.32933161573663 38.722765028166016 39.72690529260958 2018-12-01 01:16:24 2018-12-01 03:16:24
***SIGNALS: (1115, 12)
timeWindow: 2018-12-29 01:25:42 - 2018-12-29 03:25:42
timeWindow: 2018-12-29 01:25:42 - 2018-12-29 03:25:42
calling fetch_signals: 127.07321967878632 127.90456689613342 42

***SIGNALS: (1782, 12)
timeWindow: 2018-12-24 01:25:40 - 2018-12-24 03:25:40
timeWindow: 2018-12-24 01:25:40 - 2018-12-24 03:25:40
calling fetch_signals: 130.19722753463603 130.81463478257635 41.454760024161516 42.44633210281016 2018-12-24 01:25:40 2018-12-24 03:25:40
***SIGNALS: (1782, 12)
timeWindow: 2018-12-31 01:15:46 - 2018-12-31 03:15:46
timeWindow: 2018-12-31 01:15:46 - 2018-12-31 03:15:46
calling fetch_signals: 130.19722916023215 131.55009193368312 41.43643165802558 42.446330854472585 2018-12-31 01:15:46 2018-12-31 03:15:46
***SIGNALS: (1786, 12)
timeWindow: 2018-12-01 01:15:44 - 2018-12-01 03:15:44
timeWindow: 2018-12-01 01:15:44 - 2018-12-01 03:15:44
calling fetch_signals: 130.19722916023215 131.55009193368312 41.43643165802558 42.446330854472585 2018-12-01 01:15:44 2018-12-01 03:15:44
***SIGNALS: (1793, 12)
timeWindow: 2018-12-20 00:55:29 - 2018-12-20 02:55:29
timeWindow: 2018-12-20 00:55:29 - 2018-12-20 02:55:29
calling fetch_signals: 134.6327564823821 135.12031064212493 42

***SIGNALS: (2208, 12)
timeWindow: 2018-12-24 01:25:37 - 2018-12-24 03:25:37
timeWindow: 2018-12-24 01:25:37 - 2018-12-24 03:25:37
calling fetch_signals: 127.07686007036594 127.90456689613342 42.348047420448445 43.347528227711315 2018-12-24 01:25:37 2018-12-24 03:25:37
***SIGNALS: (2208, 12)
timeWindow: 2018-12-14 01:27:14 - 2018-12-14 03:27:14
timeWindow: 2018-12-14 01:27:14 - 2018-12-14 03:27:14
calling fetch_signals: 127.87495167570805 128.89331544782289 36.051668783080466 37.04662497458494 2018-12-14 01:27:14 2018-12-14 03:27:14
***SIGNALS: (2208, 12)
timeWindow: 2018-12-20 00:55:54 - 2018-12-20 02:55:54
timeWindow: 2018-12-20 00:55:54 - 2018-12-20 02:55:54
calling fetch_signals: 134.99963989303868 136.3160808918365 40.55556596196896 41.55193466744633 2018-12-20 00:55:54 2018-12-20 02:55:54
***SIGNALS: (2208, 12)
timeWindow: 2018-12-15 00:55:25 - 2018-12-15 02:55:25
timeWindow: 2018-12-15 00:55:25 - 2018-12-15 02:55:25
calling fetch_signals: 134.62292303597854 135.12031064212493 42

In [59]:
# Export results

# Set project_id to your Google Cloud Platform project ID.
project_id = 'gwu-cap'

# Set table_id to the full destination table ID. 
table_id = 'jt_data.signals_table'

#pandas_gbq.to_gbq(SIGNALS, table_id, project_id=project_id,if_exists='append')
#pandas_gbq.to_gbq(signals, table_id, project_id=project_id,if_exists='append')
pandas_gbq.to_gbq(
    SIGNALS, table_id, project_id='gwu-cap', if_exists='replace',
)





Load is 100.0% Complete




In [44]:
t1='2018-01-01'
q="""#standardSQL
SELECT
  ssvid,
  lat,
  lon,
  timestamp,
  fishing
FROM
  `global-fishing-watch.paper_dark_fishing_fleets_in_north_korea.ais_positions_2018`

WHERE
  timestamp >= '2018-12-01'
  AND timestamp < '2018-12-02'
  AND lat BETWEEN 36.0
  AND 41.6
  AND lon BETWEEN 127.7
  AND 137.0 
ORDER BY 
  timestamp, lat, lon, ssvid
"""
print("q=",q)

df = pd.read_gbq(q, project_id='gwu-cap', verbose=True, dialect='standard')
df.shape


q= #standardSQL
SELECT
  ssvid,
  lat,
  lon,
  timestamp,
  fishing
FROM
  `global-fishing-watch.paper_dark_fishing_fleets_in_north_korea.ais_positions_2018`

WHERE
  timestamp >= '2018-12-01'
  AND timestamp < '2018-12-02'
  AND lat BETWEEN 36.0
  AND 41.6
  AND lon BETWEEN 127.7
  AND 137.0 
ORDER BY 
  timestamp, lat, lon, ssvid

Requesting query... ok.
Job ID: 0ad15b1a-5d26-4c74-90a2-4383c3698465
Query running...
Query done.
Cache hit.

Retrieving results...
Got 946 rows.

Total time taken 1.04 s.
Finished at 2020-08-03 22:29:56.


(946, 5)

In [9]:

x1=params['extentLeft'] #127.0
x2=params['extentRight'] #137.0
y2=params['extentTop']   #44.0
y1=params['extentBottom']#36.0
t1="'2018-12-30'"
t2="'2018-12-31'"

print(x1,x2,y1,y2,t1,t2)

q="""#standardSQL
    SELECT 
      ssvid, lat, lon, timestamp, fishing
    FROM
      `global-fishing-watch.paper_dark_fishing_fleets_in_north_korea.ais_positions_2018` 
    WHERE 
      lon BETWEEN {x1} AND {x2}
      AND lat BETWEEN {y1} AND {y2}
      AND timestamp >= {t1}
      AND timestamp < {t2}
""".format(x1=x1,x2=x2,y1=y1,y2=y2,t1=t1,t2=t2)
print('q=',q)
df = pd.read_gbq(q, project_id='gwu-cap', verbose=True, dialect='standard')
df.size


127.0 137.0 36.0 44.0 '2018-12-30' '2018-12-31'
q= #standardSQL
    SELECT 
      ssvid, lat, lon, timestamp, fishing
    FROM
      `global-fishing-watch.paper_dark_fishing_fleets_in_north_korea.ais_positions_2018` 
    WHERE 
      lon BETWEEN 127.0 AND 137.0
      AND lat BETWEEN 36.0 AND 44.0
      AND timestamp >= '2018-12-30'
      AND timestamp < '2018-12-31'

Requesting query... ok.
Job ID: cda9e709-fde8-4f64-b6a0-b638e5d48865
Query running...
Query done.
Processed: 1.2 GB Billed: 1.2 GB
Standard price: $0.01 USD

Retrieving results...
Got 13772 rows.

Total time taken 6.26 s.
Finished at 2020-08-03 13:01:28.


68860

In [10]:
q="""#standardSQL
    SELECT 
      ssvid, lat, lon, timestamp, fishing
    FROM
      `global-fishing-watch.paper_dark_fishing_fleets_in_north_korea.ais_positions_2018` 
    WHERE 
      lat BETWEEN {y1} AND {y2}
      AND lon BETWEEN {x1} AND {x2}
      AND timestamp >= '2018-12-30'
      AND timestamp < '2018-12-31'
"""

df = pd.read_gbq(q, project_id='gwu-cap', verbose=True, dialect='standard')

if (len(df)==0):
  print('No data...')
else:
  print("Pings: ", len(df))

Requesting query... ok.
Job ID: c61e423c-fd70-4fba-8c13-f5db11388cf8
Query running...
Query done.
Processed: 0.0 B Billed: 0.0 B
Standard price: $0.00 USD

Retrieving results...


GenericGBQException: Reason: 400 Syntax error: Unexpected "{" at [7:19]

In [ ]:
%%bigquery PINGS 


In [ ]:
%matplotlib inline

In [ ]:
total_births.plot(kind="bar", x="year", y="birth_count")